## Recurrent Neural Networks

In [1]:
# FIX ALL THE RANDOM VALUES TO GET REPRODUCIBLE RESULTS
import os
os.environ["PYTHONHASHSEED"] = "0"

import random as rn
rn.seed(1254)

import numpy as np
np.random.seed(1515)

# SET NICE PLOTTING
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

import h5py
import pickle

In [2]:
plt.rcParams['figure.figsize'] = (14, 6)

In [3]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

import tensorflow as tf
tf.set_random_seed(2)

config = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1 
)

config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
sess = tf.Session(graph= tf.get_default_graph(), config=config) 
    
import keras 
from keras import backend as K
K.set_session(sess)  # set this TensorFlow session as the default session for Keras

from keras.layers import Input, Conv2D, MaxPooling2D, LSTM, GRU, Bidirectional
from keras.layers import Dense, Flatten
from keras.layers.core import Dropout
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [4]:
#SET PATHS
workspace = '/home/speakers/isobieraj/workspace'

hdf5_path = os.path.join(workspace,'dataset.hdf5') 
modelfolder = os.path.join(workspace,'models')
scalerpath = os.path.join(workspace,'scaler.pkl')

In [5]:
# LOAD DATA
hf = h5py.File(hdf5_path, 'r')
X_train = np.array(hf.get('X_train') )
y_train = np.array(hf.get('y_train'))
X_val =  np.array(hf.get('X_val'))
y_val= np.array(hf.get('y_val'))
X_test = np.array(hf.get('X_test'))
y_test = np.array(hf.get('y_test'))

n_features= hf.get('n_features').value
n_frames=hf.get('max_length_samp').value
label_list= hf.get('label_list').value

In [6]:
# SCALE THE DATA
scaler = pickle.load(open(scalerpath, 'rb'), encoding='latin1') #python2 object to python3

#scaler=pickle.load(open(scalerpath, 'rb'))

X_train_scaled = [scaler.transform(x.T) for x in X_train]
X_val_scaled = [scaler.transform(x.T) for x in X_val]
X_test_scaled = [scaler.transform(x.T) for x in X_test]

In [7]:
def build_rnn(n_frames=n_frames, n_features=n_features,
                     n_classes=10):

    # INPUT
    x = Input(shape=(n_frames, n_features), dtype='float32')
  
    y= LSTM(20, return_sequences = True)(x)
    y= Dropout(0.5)(y)  
    
    y= LSTM(20)(y)
    y= Dropout(0.5)(y) 

    y = Dense(n_classes, activation='softmax')(y)

    m = Model(inputs=x, outputs=y)
    return m

In [8]:
m = build_rnn(n_features=40,)
m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 173, 40)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 173, 20)           4880      
_________________________________________________________________
dropout_1 (Dropout)          (None, 173, 20)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 20)                3280      
_________________________________________________________________
dropout_2 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                210       
Total params: 8,370
Trainable params: 8,370
Non-trainable params: 0
_________________________________________________________________


In [9]:
from keras.utils import plot_model
plot_model(m, to_file='figures/rnn_model_shape.png', show_shapes=True)

<img src='figures/rnn_model_shape.png'> 

In [ ]:
adam= keras.optimizers.Adam(lr=0.001, decay=1e-5)
m.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
model_checkpoint = ModelCheckpoint(
            os.path.join(modelfolder, 
                    'rnn_epoch_{epoch:03d}_val_loss_{val_loss:.4f}.hdf5'),
            monitor='val_loss', 
            save_best_only=True)

early_stopping = EarlyStopping(monitor='val_loss', patience=50, verbose=1)
callbacks = [early_stopping, model_checkpoint]

history = m.fit(x=np.array(X_train_scaled), y=y_train, batch_size=130,
                    epochs=300, verbose=True,
                    validation_data=(np.array(X_val_scaled), y_val), shuffle=True)

Train on 7079 samples, validate on 816 samples
Epoch 1/300
7079/7079 [==============================] - 16s 2ms/step - loss: 2.2088 - acc: 0.1721 - val_loss: 2.0274 - val_acc: 0.2635
Epoch 2/300
7079/7079 [==============================] - 15s 2ms/step - loss: 2.0249 - acc: 0.2431 - val_loss: 1.8886 - val_acc: 0.2537
Epoch 3/300
7079/7079 [==============================] - 15s 2ms/step - loss: 1.9281 - acc: 0.2705 - val_loss: 1.8063 - val_acc: 0.2941
Epoch 4/300
7079/7079 [==============================] - 16s 2ms/step - loss: 1.8037 - acc: 0.3224 - val_loss: 1.6423 - val_acc: 0.3272
Epoch 5/300
7079/7079 [==============================] - 16s 2ms/step - loss: 1.7459 - acc: 0.3633 - val_loss: 1.6612 - val_acc: 0.3150
Epoch 6/300
7079/7079 [==============================] - 16s 2ms/step - loss: 1.7785 - acc: 0.3618 - val_loss: 1.7074 - val_acc: 0.2745
Epoch 7/300
7079/7079 [==============================] - 16s 2ms/step - loss: 1.7342 - acc: 0.3666 - val_loss: 1.5701 - val_acc: 0.3701
E

7079/7079 [==============================] - 17s 2ms/step - loss: 1.4948 - acc: 0.4704 - val_loss: 1.4938 - val_acc: 0.4816
Epoch 61/300
7079/7079 [==============================] - 17s 2ms/step - loss: 1.7551 - acc: 0.4212 - val_loss: 1.8067 - val_acc: 0.4314
Epoch 62/300
7079/7079 [==============================] - 17s 2ms/step - loss: 1.8225 - acc: 0.3645 - val_loss: 1.7696 - val_acc: 0.3382
Epoch 63/300
7079/7079 [==============================] - 17s 2ms/step - loss: 1.6765 - acc: 0.4027 - val_loss: 1.6362 - val_acc: 0.3787
Epoch 64/300
7079/7079 [==============================] - 17s 2ms/step - loss: 1.6240 - acc: 0.4174 - val_loss: 1.5887 - val_acc: 0.3922
Epoch 65/300
7079/7079 [==============================] - 17s 2ms/step - loss: 1.5797 - acc: 0.4347 - val_loss: 1.6039 - val_acc: 0.3958
Epoch 66/300
7079/7079 [==============================] - 17s 2ms/step - loss: 1.5540 - acc: 0.4570 - val_loss: 1.6037 - val_acc: 0.3848
Epoch 67/300
7079/7079 [==============================

Epoch 120/300
7079/7079 [==============================] - 17s 2ms/step - loss: 1.2903 - acc: 0.5577 - val_loss: 1.5048 - val_acc: 0.4951
Epoch 121/300
7079/7079 [==============================] - 18s 2ms/step - loss: 1.3042 - acc: 0.5563 - val_loss: 1.4020 - val_acc: 0.5319
Epoch 122/300
7079/7079 [==============================] - 17s 2ms/step - loss: 1.3936 - acc: 0.5386 - val_loss: 1.4234 - val_acc: 0.5221
Epoch 123/300
7079/7079 [==============================] - 17s 2ms/step - loss: 1.4872 - acc: 0.4841 - val_loss: 1.4724 - val_acc: 0.4804
Epoch 124/300
7079/7079 [==============================] - 17s 2ms/step - loss: 1.3823 - acc: 0.5156 - val_loss: 1.5018 - val_acc: 0.4828
Epoch 125/300
7079/7079 [==============================] - 17s 2ms/step - loss: 1.3225 - acc: 0.5495 - val_loss: 1.4566 - val_acc: 0.5233
Epoch 126/300
7079/7079 [==============================] - 17s 2ms/step - loss: 1.2715 - acc: 0.5642 - val_loss: 1.4229 - val_acc: 0.5306
Epoch 127/300
7079/7079 [=========

7079/7079 [==============================] - 17s 2ms/step - loss: 1.1281 - acc: 0.6371 - val_loss: 1.4384 - val_acc: 0.5441
Epoch 180/300
7079/7079 [==============================] - 17s 2ms/step - loss: 1.1078 - acc: 0.6471 - val_loss: 1.3473 - val_acc: 0.5723
Epoch 181/300
7079/7079 [==============================] - 17s 2ms/step - loss: 1.0913 - acc: 0.6481 - val_loss: 1.4514 - val_acc: 0.5478
Epoch 182/300
7079/7079 [==============================] - 17s 2ms/step - loss: 1.0588 - acc: 0.6620 - val_loss: 1.4407 - val_acc: 0.5564
Epoch 183/300
7079/7079 [==============================] - 17s 2ms/step - loss: 1.0509 - acc: 0.6564 - val_loss: 1.5097 - val_acc: 0.5306
Epoch 184/300
7079/7079 [==============================] - 17s 2ms/step - loss: 1.1539 - acc: 0.6288 - val_loss: 1.5298 - val_acc: 0.5343
Epoch 185/300
7079/7079 [==============================] - 17s 2ms/step - loss: 1.6102 - acc: 0.5012 - val_loss: 1.5474 - val_acc: 0.4951
Epoch 186/300
7079/7079 [=======================

In [ ]:
plt.subplot(1,2,1)
plt.plot(history.history['acc'], label='Accuracy')
plt.plot(history.history['val_acc'], label='Validation accuracy')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Loss')

In [ ]:
prediction = m.predict(np.array(X_test_scaled), batch_size=130, verbose=1)
y_predict=np.array([ np.argmax(p) for p in prediction])
y_test_label = np.array([np.argmax(y) for y in y_test])

In [ ]:
from sklearn.metrics import confusion_matrix

acc = np.sum(y_test_label==y_predict) / float(len(y_test_label))
print('Accuracy: {:.2f}'.format(acc))

cm = confusion_matrix(y_test_label, y_predict )
sns.heatmap(cm, annot=True, annot_kws={"size": 14}, fmt='g', linewidths=.5, 
            yticklabels=label_list,xticklabels=label_list)
plt.ylabel('True label')
plt.xlabel('Predicted label')